In [ ]:
"""
Завдання
Уявімо, що у нас є компанія, яка виробляє два види продуктів: A та B.
Для виробництва цих продуктів використовуються три види ресурсів:
ресурс 1, ресурс 2 та ресурс 3. Кожен продукт приносить певний прибуток,
і ми хочемо максимізувати загальний прибуток.

Вхідні дані:
Продукт A приносить прибуток $3 за одиницю, продукт B - $5 за одиницю.
Кількість ресурсів:
Ресурс 1: 4 одиниці.
Ресурс 2: 12 одиниць.
Ресурс 3: 18 одиниць.
Продукт A потребує:
Ресурс 1: 1 одиницю на одиницю продукту.
Ресурс 2: 3 одиниці на одиницю продукту.
Ресурс 3: 2 одиниці на одиницю продукту.
Продукт B потребує:
Ресурс 1: 2 одиниці на одиницю продукту.
Ресурс 2: 2 одиниці на одиницю продукту.
Ресурс 3: 4 одиниці на одиницю продукту.
Постановка задачі:
Ми хочемо знайти оптимальні кількості продуктів A і B, які потрібно виробити,
щоб максимізувати загальний прибуток, не перевищуючи доступні ресурси.

Максимізувати: 3A + 5B
З обмеженнями:
1) 1A+2B≤4 -> 4 одиниці ресурсу 1
2) 3A+2B≤12 -> 12 однициь ресурсу 2
3) 2A+4B≤18 -> 18 одиниць ресурсу 3
4) A≥0,B≥0 -> виробляємо саме продукти А та Б, без "залізання в борги"
"""


import numpy as np

def simplex(c, A, b):
    """
    Реалізація симплекс-методу для задачі лінійного програмування
    """
    m, n = A.shape

    # Створюємо таблицю симплекс-методу
    tableau = np.zeros((m+1, n+1))
    tableau[:-1, :-1] = A
    tableau[:-1, -1] = b
    tableau[-1, :-1] = -c

    # Функція для знаходження стовпця з найменшим значенням у останньому рядку
    def get_pivot_column():
        return np.argmin(tableau[-1, :-1])

    # Функція для знаходження рядка з мінімальним відношенням останнього стовпця до відповідного елементу в обраному стовпці
    def get_pivot_row(pivot_col):
        rows = np.where(tableau[:-1, pivot_col] > 0)[0]
        return rows[np.argmin(tableau[rows, -1] / tableau[rows, pivot_col])]

    # Поки є від'ємні значення у останньому рядку, продовжуємо ітерації
    while np.min(tableau[-1, :-1]) < 0:
        pivot_col = get_pivot_column()
        pivot_row = get_pivot_row(pivot_col)

        pivot_value = tableau[pivot_row, pivot_col]
        tableau[pivot_row] /= pivot_value

        for row in range(m+1):
            if row != pivot_row:
                tableau[row] -= tableau[row, pivot_col] * tableau[pivot_row]

    # Оптимальні значення змінних
    x = np.zeros(n)
    for i in range(n):
        col = tableau[:-1, i]
        if np.sum(col == 1) == 1 and np.sum(col == 0) == m-1:
            x[i] = tableau[np.where(col == 1)[0][0], -1]

    # Оптимальне значення цільової функції
    max_value = tableau[-1, -1]

    return np.round(x).astype(int), int(round(max_value))

# Вхідні дані
c = np.array([3, 5])
A = np.array([
    [1, 2],
    [3, 2],
    [2, 4]
])
b = np.array([8, 12, 18])

# Розв'язання задачі симплекс методом
x, max_value = simplex(c, A, b)

print(x, max_value)

print(f"Оптимальна кількість продукту A: {x[0]}")
print(f"Оптимальна кількість продукту B: {x[1]}")
print(f"Максимальний прибуток: {max_value}")

In [ ]:

import pulp

# Ініціалізація моделі
model = pulp.LpProblem("Maximize Profit", pulp.LpMaximize)

# Визначення змінних
A = pulp.LpVariable('A', lowBound=0, cat='Integer')  # Кількість продукту A
B = pulp.LpVariable('B', lowBound=0, cat='Integer')  # Кількість продукту B

# Функція цілі (Максимізація прибутку)
model += 3 * A + 5 * B, "Profit"

# Додавання обмежень
model += 1 * A + 2 * B <= 8   # Обмеження по ресурсу 1
model += 3 * A + 2 * B <= 12  # Обмеження по ресурсу 2
model += 2 * A + 4 * B <= 18  # Обмеження по ресурсу 3

# Розв'язання моделі
model.solve()

# Вивід результатів
print("Виробляти продуктів A:", A.varValue)
print("Виробляти продуктів B:", B.varValue)
print("Максимальний прибуток:", pulp.value(model.objective))

In [ ]:
import random

def is_prime(n, k=5):
    """Перевірка простоти числа n за допомогою алгоритму Рабіна-Міллера."""

    # Якщо n < 2, воно не може бути простим
    if n < 2:
        return False
    # Якщо n == 2 або n == 3, воно є простим
    if n in (2, 3):
        return True
    # Якщо n парне, воно не є простим
    if n % 2 == 0:
        return False

    # 1) Представляємо число n як n-1 = 2^r * d, де d — непарне число.
    r, d = 0, n - 1
    while d % 2 == 0:
        d //= 2
        r += 1

    # def fermat_test(n, k=5):
    #     if n <= 1:
    #         return False
    #     for _ in range(k):
    #         a = random.randint(1, n - 1)
    #     if pow(a, n - 1, n) != 1:
    #         return False
    #     return True


    # Перевіряємо k випадкових баз
    for _ in range(k):
        # 2) Виберіть випадкове число a з діапазону [2, n−2].
        a = random.randint(2, n - 2)
        # 3) Обчисліть x = a^m mod n. Якщо x=1 або x=n−1, то перейдіть до наступного кроку.
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        # 4) Повторіть q разів: обчисліт x=x^2 modn. Якщо x=n−1, то перейдіть до наступного кроку.
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else: # Якщо попередній крок не вдалося завершити, то n є складеним.
            return False
    # Якщо жоден із кроків не визначив n як складене, то n ймовірно просте.
    return True



# Тестування функції
number = 61
iterations = 10
if is_prime(number, iterations):
    print(f"Число {number} ймовірно є простим.")
else:
    print(f"Число {number} є складеним.")

In [ ]:
import random

class UniversalHash:
    def __init__(self, m, max_key):
        self.m = m
        self.p = self._next_prime(max_key)
        self.a = random.randint(1, self.p - 1)
        self.b = random.randint(0, self.p - 1)

    def _next_prime(self, n):
        while True:
            n += 1
            for i in range(2, int(n ** 0.5) + 1):
                if n % i == 0:
                    break
            else:
                return n

    def hash(self, key):
        return ((self.a * key + self.b) % self.p) % self.m

# Приклад використання:
hasher = UniversalHash(100, 1000)
print(hasher.hash(123))
print(hasher.hash(456))

In [ ]:
"""
Розглянемо квадрат зі стороною 2, центр якого співпадає з центром вписаного в
нього кола радіусом 1. Якщо випадково кидати точки на площині, то відношення кількості точок,
що потрапили в коло, до загальної кількості точок буде приблизно дорівнювати
відношенню площі кола до площі квадрата, тобто π/4.
"""

import random
import matplotlib.pyplot as plt

def monte_carlo_pi(num_samples):
    # 1. Визначення моделі або системи.
    inside_circle = 0
    x_inside, y_inside = [], []
    x_outside, y_outside = [], []
    # 2. Генерація випадкових вхідних даних
    for _ in range(num_samples):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        # 3. Виконання обчислень
        if x**2 + y**2 <= 1:
            inside_circle += 1
            x_inside.append(x)
            y_inside.append(y)
        else:
            x_outside.append(x)
            y_outside.append(y)

    # 4. Агрегування та аналіз результатів
    pi_estimate = 4 * inside_circle / num_samples
    return pi_estimate, x_inside, y_inside, x_outside, y_outside

# Задаємо кількість випадкових точок
num_samples = 100000

# Запускаємо метод Монте-Карло для обчислення π
pi_estimate, x_inside, y_inside, x_outside, y_outside = monte_carlo_pi(num_samples)

# Виводимо результат
print(f"Оцінка значення π за методом Монте-Карло з {num_samples} випадкових точок: {pi_estimate}")

# Візуалізація результатів
plt.figure(figsize=(8, 8))
plt.scatter(x_inside, y_inside, color='blue', s=1, label='Точки всередині кола')
plt.scatter(x_outside, y_outside, color='red', s=1, label='Точки поза колом')
circle = plt.Circle((0, 0), 1, color='black', fill=False, linewidth=2)
plt.gca().add_patch(circle)
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.gca().set_aspect('equal', adjustable='box')
plt.title(f"Метод Монте-Карло для оцінки π\nЧисло точок: {num_samples}\nОцінка π: {pi_estimate}")
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Retrieve the previously defined function and limits, and calculated values
def f(x):
    return x**2 + 2*x - 6


def monte_carlo_integral(a, b, min_y, max_y, num_samples):
    # 1. Визначення моделі або системи.
    inside_circle = 0
    x_inside, y_inside = [], []
    x_outside, y_outside = [], []
    # 2. Генерація випадкових вхідних даних
    for _ in range(num_samples):
        x = random.uniform(a, b)
        y = random.uniform(min_y, max_y)
        # 3. Виконання обчислень
        if f(x) >= y:
            inside_circle += 1
            x_inside.append(x)
            y_inside.append(y)
        else:
            x_outside.append(x)
            y_outside.append(y)

    # 4. Агрегування та аналіз результатів
    integral_estimate = (b - a) * (max_y - min_y) * (inside_circle /num_samples)
    return integral_estimate, x_inside, y_inside, x_outside, y_outside

plt.figure(figsize=(10, 6))


a = 0
b = 8
min_y = -6;
max_y = 100
num_samples = 10000
integral_estimate, x_inside, y_inside, x_outside, y_outside = monte_carlo_integral(a, b, min_y, max_y, num_samples)
print(integral_estimate)
x_plot = np.linspace(a, b, 500)
y_plot = [f(val) for val in x_plot]

plt.plot(x_plot, y_plot, color='blue', label='f(x) = x^2')


plt.scatter(x_inside, y_inside, color='blue', s=1, label='Точки під графіком')
plt.scatter(x_outside, y_outside, color='red', s=1, label='Точки над колом')

# Fill the area under the curve for visualization of the integral
plt.fill_between(x_plot, 0, y_plot, color='blue', alpha=0.1)

plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.grid(True)
plt.show()